### Data Processing

In [153]:
import os
import pandas as pd
import numpy as np

start_year = 1992
end_year = 2022
years_range = list(range(start_year, end_year + 1))

methane_df = pd.read_csv("/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_processeddata/CH4_raw_na.csv")
country = methane_df["country"].values # list of countries in total


##### CHANGE THE CODE
# get all independent variables from 1992 - 2022, add median to missing values
def get_country_data(years_range, country_name):
    folder_path = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_processeddata"
    # get the list of files in the folder
    file_names = os.listdir(folder_path)
    # print(file_names)
    column_names = ['Abbreviation'] + years_range
    # print(column_names)
    # create a new dataframe for each country
    country_df = pd.DataFrame(columns=column_names)
    
    # iterate through features
    for file in file_names:
        if file in ["CH4_raw_na.csv", "PRS_na.csv", "PRS_raw_na.csv"]:
            continue
        file_path = os.path.join(folder_path, file)
        # print(file_path)
        df = pd.read_csv(file_path)
        # print(df)
        columns = df.columns.tolist()
        columns[3:] = [col[-4:] for col in columns[3:]]
        df.columns = columns
        # start year for each feature
        starting_year = int(df.columns[3])
        # not including features that only have values after 2022
        if starting_year > end_year:
            print(f"Starting year {starting_year} not found in country_df columns.")
            continue
        # print(starting_year)
        for i, item in df.iterrows():
            if item["country"] == country_name:
                if file[:3] not in country_df['Abbreviation'].values:
                    # Add a new row with the file abbreviation
                    new_row = pd.Series([file[:3]] + [None] * (len(column_names) - 1), index=column_names)
                    # start index of the df
                    if starting_year <= years_range[0]:
                        start_index = df.columns.get_loc(str(years_range[0]))
                    else:
                        start_index = 3
                    values_to_insert = item[start_index:].values
                    #print(len(values_to_insert))
                    if len(values_to_insert) == 0:
                        continue
                    # start index of the df
                    if starting_year <= years_range[0]:
                        starting_index = 1
                    else:
                        starting_index = country_df.columns.get_loc(starting_year)
                    #print(starting_index)
                    ending_index = starting_index + len(values_to_insert)
                    if ending_index > len(new_row):
                        ending_index = len(new_row)
                    #print(ending_index)
                    # insert values into the new row
                    # Ensure that the length of values_to_insert fits the slice
                    if len(values_to_insert) > (ending_index - starting_index):
                        values_to_insert = values_to_insert[:(ending_index - starting_index)]
                        
                    try:
                        new_row.iloc[starting_index:ending_index] = values_to_insert
                    except Exception as e:
                        print(f"Error during insertion: {e}")
                        print(f"Values to insert: {values_to_insert}")
                        print(f"Slice start_index:end_index: {starting_index}:{ending_index}")
                        print(f"New row after error: {new_row}")
                        raise
                    
                    # print(new_row)
                    new_row_df = new_row.to_frame().transpose()
                    country_df = pd.concat([country_df, new_row_df], ignore_index=True)
                    #print(country_df)
                break
        
    country_df = country_df.replace({None: pd.NA})
    # Convert relevant columns to numeric
    for col in country_df.columns[1:]:  # Exclude 'Abbreviation' column
        country_df[col] = pd.to_numeric(country_df[col], errors='coerce')
    numerical_df = country_df.select_dtypes(include='number')
    medians = numerical_df.median()
    # Fill NaN values with the median of each numeric column
    country_df[numerical_df.columns] = numerical_df.fillna(medians)
    country_df_transposed = country_df.set_index('Abbreviation').transpose()
    return country_df_transposed

def input_data_to_excel(country_list):
    start_year = 1992
    end_year = 2022
    years_range = list(range(start_year, end_year + 1))
    for c in country_list:
        country_df_transposed = get_country_data(years_range, c)
        file_path = f"/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata/{c}.xlsx"
        country_df_transposed.to_excel(file_path, index=False)

# input the data into excel
# input_data_to_excel(country)

In [154]:
# Transfer methane dataframe
for col in methane_df.columns[3:]: 
    methane_df[col] = pd.to_numeric(methane_df[col], errors='coerce')
    num_df = methane_df.select_dtypes(include='number')
    medians = num_df.median()
    # Fill NaN values with the median of each numeric column
    methane_df[num_df.columns] = num_df.fillna(medians)

methane_df_last_31 = methane_df.iloc[:, -31:]
methane_df_3 = methane_df.iloc[:, 2]
methane_df = pd.concat([methane_df_3, methane_df_last_31], axis=1)
methane_df.head()

,country,CH4.raw.1992,CH4.raw.1993,CH4.raw.1994,CH4.raw.1995,CH4.raw.1996,CH4.raw.1997,CH4.raw.1998,CH4.raw.1999,CH4.raw.2000,...,CH4.raw.2013,CH4.raw.2014,CH4.raw.2015,CH4.raw.2016,CH4.raw.2017,CH4.raw.2018,CH4.raw.2019,CH4.raw.2020,CH4.raw.2021,CH4.raw.2022
0,Afghanistan,301.00,308.00,322.00,341.00,377.00,407.00,431.00,465.00,418.00,...,650.0,667.00,644.00,646.00,647.00,665.0,667.00,645.00,668.00,670.00
1,Albania,119.00,124.00,146.00,145.00,138.00,128.00,123.00,125.00,127.00,...,107.0,109.00,110.00,109.00,108.00,109.0,100.00,93.70,92.50,90.20
2,Algeria,2500.00,2670.00,2650.00,2920.00,3000.00,2830.00,2780.00,2880.00,3070.00,...,2870.0,2980.00,3000.00,3170.00,3170.00,3180.0,3090.00,2960.00,3210.00,3270.00
3,Andorra,3.12,3.18,3.25,3.33,3.37,3.34,3.33,3.32,3.43,...,3.2,3.14,3.08,3.09,3.09,3.1,3.05,3.02,3.08,2.86
4,Angola,1110.00,1230.00,1630.00,1820.00,2030.00,2080.00,2270.00,2230.00,2130.00,...,2170.0,2120.00,2220.00,2200.00,2170.00,2020.0,2010.00,1920.00,1830.00,1760.00


In [155]:
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from statsmodels.regression.linear_model import OLS
from zipfile import BadZipFile

### Check Stationary for Y

In [163]:
def test_stationarity(series):
    result = adfuller(series)
    p_value = result[1]  # p-value from the ADF test
    return p_value

methane_df_t = methane_df.T
methane_df_t.columns = methane_df_t.iloc[0]
methane_df_t = methane_df_t.drop(methane_df_t.index[0])

# print(methane_df_t.head())

columns_to_drop = []
# Apply the ADF test to each row and store results
for i in range(len(methane_df_t.columns)):
    if test_stationarity(methane_df_t.iloc[:, i]) > 0.5:
        column_name = methane_df_t.columns[i]
        # print(column_name)
        methane_df_t[column_name] = pd.to_numeric(methane_df_t.iloc[:, i], errors='coerce')

        # Create cube root and differenced columns
        methane_df_t[f'{column_name}_cbrt'] = np.cbrt(methane_df_t.iloc[:, i])
        methane_df_t[f'{column_name}_cbrt_diff'] = methane_df_t[f'{column_name}_cbrt'].diff()
        first_value = methane_df_t[f'{column_name}_cbrt_diff'].iloc[1]
        methane_df_t[f'{column_name}_cbrt_diff'] = methane_df_t[f'{column_name}_cbrt_diff'].fillna(first_value)
        columns_to_drop.append(column_name)
        columns_to_drop.append(f'{column_name}_cbrt')
        
methane_df_t.drop(columns=columns_to_drop, axis=1, inplace=True)

<ipython-input-163-fabd116204c4>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  methane_df_t[f'{column_name}_cbrt_diff'] = methane_df_t[f'{column_name}_cbrt'].diff()
<ipython-input-163-fabd116204c4>:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  methane_df_t[f'{column_name}_cbrt'] = np.cbrt(methane_df_t.iloc[:, i])
<ipython-input-163-fabd116204c4>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

220

In [164]:
methane_df_t

country,Algeria,Angola,Anguilla,Argentina,Armenia,Aruba,Australia,Austria,Belarus,Bermuda,...,Tonga_cbrt_diff,Turkiye_cbrt_diff,Turkmenistan_cbrt_diff,Tuvalu_cbrt_diff,Uganda_cbrt_diff,United Arab Emirates_cbrt_diff,Uruguay_cbrt_diff,Uzbekistan_cbrt_diff,Venezuela_cbrt_diff,Zambia_cbrt_diff
CH4.raw.1992,2500.0,1110.0,0.0935,4460.0,54.7,0.802,5640.0,522.0,681.0,301.0,...,-0.001521,-0.051104,0.431033,-0.004673,0.074261,0.399250,0.098393,0.197440,0.085135,-0.015317
CH4.raw.1993,2670.0,1230.0,0.097,4450.0,47.4,0.846,5560.0,508.0,648.0,306.0,...,-0.001521,-0.051104,0.431033,-0.004673,0.074261,0.399250,0.098393,0.197440,0.085135,-0.015317
CH4.raw.1994,2650.0,1630.0,0.101,4510.0,47.2,0.893,5540.0,494.0,618.0,301.0,...,-0.007651,-0.051566,-1.293388,0.003899,0.004903,-0.095296,0.023305,0.322819,0.166891,-0.020514
CH4.raw.1995,2920.0,1820.0,0.104,4590.0,46.0,0.929,5520.0,476.0,587.0,320.0,...,-0.021858,0.153322,-0.068304,0.000774,0.053557,0.188860,-0.019413,0.026080,0.399128,-0.015454
CH4.raw.1996,3000.0,2030.0,0.106,4470.0,46.2,0.941,5290.0,467.0,571.0,338.0,...,0.003163,0.342687,0.210057,0.000000,0.081435,0.211997,0.046455,0.154012,0.115121,-0.125834
CH4.raw.1997,2830.0,2080.0,0.108,4480.0,46.2,0.976,5330.0,456.0,558.0,346.0,...,0.000000,0.368810,-1.343562,0.003079,0.107719,-0.120089,-0.038680,-0.050877,0.224419,0.063413
CH4.raw.1998,2780.0,2270.0,0.11,4390.0,44.3,0.991,5450.0,446.0,555.0,355.0,...,0.000000,0.132720,0.341637,0.000765,0.123063,-0.122907,-0.070450,0.151298,0.252709,0.214371
CH4.raw.1999,2880.0,2230.0,0.112,4550.0,44.7,1.0,6030.0,442.0,550.0,355.0,...,0.009407,0.215061,1.063070,0.003043,-0.013500,0.000000,0.019678,-0.176907,-0.125180,-0.100665
CH4.raw.2000,3070.0,2130.0,0.116,4610.0,45.0,2.85,6430.0,435.0,528.0,374.0,...,0.012358,0.248719,1.025544,0.000756,0.049296,0.213230,-0.063267,0.176907,0.213214,-0.071985
CH4.raw.2001,2730.0,2080.0,0.119,4570.0,45.8,2.87,6870.0,431.0,520.0,356.0,...,0.000000,0.040525,0.064947,0.001508,0.053144,0.059368,-0.003983,0.408508,-0.017517,0.056667


In [185]:
methane_df_t.shape

(31, 220)

### Check Stationary for X

In [147]:
sample = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata/Afghanistan.xlsx"
sample_df = pd.read_excel(sample)
feature = list(sample_df.columns)

In [61]:
# Define the folder path and retrieve file names
folder_path_country = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata/"
file_names_country = os.listdir(folder_path_country)

# Initialize an empty DataFrame to store ADF p-values
adf_results = pd.DataFrame(columns=feature)

for file in file_names_country:
    file_path = os.path.join(folder_path_country, file)
    
    try:
        # Attempt to read the Excel file
        df_country = pd.read_excel(file_path, engine='openpyxl')
    except BadZipFile:
        print(f"Skipping file {file}: Not a valid zip file")
        continue
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    
    # List to store p-values for the current file (country)
    adf_p = []
    country_name = os.path.splitext(file)[0]
    
    for column in df_country.columns:
        try:
            # Check if the column is constant
            if df_country[column].nunique() <= 1:
                print(f"Skipping column {column} in {file}: Column is constant")
                adf_p.append(None)  # Append None if the column is constant
                continue
            
            # Apply ADF test and store the p-value
            p_value = adfuller(df_country[column])[1]
            adf_p.append(p_value)
            
            # If p-value > 0.5, apply differencing
            if p_value > 0.5:
                df_country[f'{column}_cbrt'] = np.cbrt(df_country[column])
                df_country[f'{column}_cbrt_diff'] = df_country[f'{column}_cbrt'].diff()
                first_value = df_country[f'{column}_cbrt_diff'].iloc[1]
                df_country[f'{column}_cbrt_diff'] = df_country[f'{column}_cbrt_diff'].fillna(first_value)
        except ValueError as ve:
            print(f"Skipping column {column} in {file}: {ve}")
            adf_p.append(None)  # Append None if there is a ValueError
            continue
    
    # Add the country name and p-values to the DataFrame
    adf_results.loc[country_name] = adf_p
    
    # Group columns by the first three characters
    grouped_cols = df_country.columns.to_series().groupby(df_country.columns.to_series().str[:3])

    # Identify columns to keep
    columns_to_keep = []
    for group, cols in grouped_cols:
        # If there's only one column in the group, keep it
        if len(cols) == 1:
            columns_to_keep.append(cols[0])
        else:
            # Filter out columns that end with '_cbrt_diff' if there are multiple columns
            cbrt_diff_cols = [col for col in cols if col.endswith('_cbrt_diff')]
            if cbrt_diff_cols:
                columns_to_keep.append(cbrt_diff_cols[0])
                
    df_filtered = df_country[columns_to_keep]

    file_path_new = f"/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata_new/{country_name}.xlsx"
    df_filtered.to_excel(file_path_new, index=False)

# Display the first few rows of the results
# print(adf_results.head())


Skipping column SPI in Micronesia.xlsx: Column is constant
Skipping column TBN in Micronesia.xlsx: Column is constant
Skipping column SPI in Saint Vincent and the Grenadines.xlsx: Column is constant
Skipping column TBN in Saint Vincent and the Grenadines.xlsx: Column is constant


/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Applications/Mu Editor.app/Cont

Skipping column TBN in Somalia.xlsx: Column is constant


/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


Skipping column TBN in Brunei Darussalam.xlsx: Column is constant


...

/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


Skipping column TBN in Cook Islands.xlsx: Column is constant
Skipping column TBN in Monaco.xlsx: Column is constant


<ipython-input-61-4096836043b0>:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_country[f'{column}_cbrt'] = np.cbrt(df_country[column])
<ipython-input-61-4096836043b0>:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_country[f'{column}_cbrt_diff'] = df_country[f'{column}_cbrt'].diff()
<ipython-input-61-4096836043b0>:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

Skipping column TBN in Greenland.xlsx: Column is constant
Skipping column BLC in Greenland.xlsx: Column is constant


/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


In [148]:
adf_results_true_false = adf_results <= 0.5
adf_results_true_false.head()

,PDN,VOE,CBP,HPE,LED,TC5,LUF,RMS,NXA,FCL,...,SOE,GHI,GTP,MHP,POP,CDO,FSS,MPE,FOG,PAE
Sweden,True,True,True,False,False,False,True,True,True,True,...,True,False,True,False,False,False,False,True,True,False
Guernsey,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
Micronesia,True,True,False,True,False,True,True,False,False,True,...,True,True,True,True,False,True,False,False,True,True
Oman,True,True,False,True,False,False,False,True,True,False,...,True,True,False,False,False,False,False,True,False,False
Saint Vincent and the Grenadines,False,False,True,True,False,True,True,False,True,True,...,False,True,False,False,True,True,True,True,True,False


### Feature Importance Models

#### Lasso

In [149]:
from sklearn.linear_model import LassoCV

def lasso(x_train_country, y_train_country):
    model = LassoCV(cv=5).fit(x_train_country, y_train_country)
    importance = model.coef_.tolist()
    return importance

In [166]:
folder_path_country = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata_new/"
file_names_country = os.listdir(folder_path_country)

# for each country
lasso_df = pd.DataFrame(columns=feature)
for file in file_names_country:
    file_path = os.path.join(folder_path_country, file)
    try:
        # Attempt to read the Excel file
        df_country = pd.read_excel(file_path, engine='openpyxl')
    except BadZipFile:
        print(f"Skipping file {file}: Not a valid zip file")
        continue
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    country_name = os.path.splitext(file)[0]
    for c in methane_df_t.columns:
        if country_name in c:
            y_country = methane_df_t[c].values
            break
    X_train, X_test, y_train, y_test = train_test_split(df_country, y_country, test_size=0.2, random_state=42)
    feature_rank = lasso(X_train, y_train)
    new_row_df = pd.Series(feature_rank, index=feature)
    new_row_df = new_row_df.to_frame().T
    lasso_df = pd.concat([lasso_df, new_row_df], axis = 0,ignore_index=True)
lasso_df

/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.450e+00, tolerance: 2.147e+00
  model = cd_fast.enet_coordinate_descent(
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.485e+00, tolerance: 2.147e+00
  model = cd_fast.enet_coordinate_descent(
/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number o

,PDN,VOE,CBP,HPE,LED,TC5,LUF,RMS,NXA,FCL,...,SOE,GHI,GTP,MHP,POP,CDO,FSS,MPE,FOG,PAE
0,0.0,0.000000,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0000,0.0,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-15.124895,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.0,0.000000,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0000,-0.0,...,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0
4,0.0,0.000000,-0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0000,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,-0.0,-0.000000,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0587,-0.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
216,-0.0,-0.000000,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0000,-0.0,...,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
217,-0.0,-0.000000,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0000,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
218,-0.0,-0.000000,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0000,-0.0,...,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [167]:
lasso_feature_rank = abs(lasso_df.mean()).sort_values(ascending=False)
lasso_feature = list(lasso_feature_rank.head(30).index)
lasso_feature

['GDP',
 'TCG',
 'CDA',
 'MKP',
 'BLC',
 'OEC',
 'PDN',
 'CBP',
 'PMD',
 'SOE',
 'BCA',
 'PFL',
 'WWT',
 'RCY',
 'HPE',
 'WPC',
 'OZD',
 'MHP',
 'VOE',
 'GHG',
 'OEB',
 'PHL',
 'NXA',
 'SO2',
 'TCA',
 'BER',
 'GHN',
 'CHA',
 'BTZ',
 'GDB']

#### Ridge

In [171]:
from sklearn.linear_model import RidgeCV

folder_path_country = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata_new/"
file_names_country = os.listdir(folder_path_country)

# Initialize DataFrame for Ridge results
ridge_df = pd.DataFrame(columns=feature)

for file in file_names_country:
    file_path = os.path.join(folder_path_country, file)
    try:
        # Attempt to read the Excel file
        df_country = pd.read_excel(file_path, engine='openpyxl')
    except BadZipFile:
        print(f"Skipping file {file}: Not a valid zip file")
        continue
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    
    country_name = os.path.splitext(file)[0]
    
    for c in methane_df_t.columns:
        if country_name in c:
            y_country = methane_df_t[c].values
            break
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(df_country, y_country, test_size=0.2, random_state=42)
    
    # Fit Ridge regression model
    ridge_model = RidgeCV(alphas=[0.1, 1.0, 10.0], store_cv_values=True)  # Adjust alphas as needed
    ridge_model.fit(X_train, y_train)
    
    # Get the coefficients
    feature_rank = ridge_model.coef_
    
    # Convert to DataFrame and append
    new_row_df = pd.Series(feature_rank, index=feature)
    new_row_df = new_row_df.to_frame().T
    ridge_df = pd.concat([ridge_df, new_row_df], axis=0, ignore_index=True)

ridge_df

,PDN,VOE,CBP,HPE,LED,TC5,LUF,RMS,NXA,FCL,...,SOE,GHI,GTP,MHP,POP,CDO,FSS,MPE,FOG,PAE
0,0.003053,-0.002092,-2.601115e-04,0.001651,0.001713,-0.004902,0.003450,0.007859,-0.014691,0.003467,...,-0.005327,8.079256e-04,0.000314,0.003014,-0.008062,-0.002723,-0.002677,-0.001620,-0.001620,-0.003946
1,1.120675,0.523302,1.120675e+00,1.120675,1.120675,1.120675,1.120675,1.120675,1.120675,1.120675,...,1.120675,1.120675e+00,1.120675,-0.740189,1.120675,1.120675,1.120675,1.120675,1.120675,1.120675
2,-0.002410,-0.001050,9.116725e-05,0.001239,0.001324,-0.001611,-0.002418,-0.007428,-0.006944,-0.002896,...,-0.001617,-7.804157e-03,0.008415,-0.002189,-0.000518,-0.002844,-0.001504,-0.002844,-0.002844,-0.002844
3,-0.000236,0.009950,-2.202309e-02,0.002745,0.002836,0.002530,-0.001795,0.010215,-0.012074,-0.001836,...,-0.002215,-1.056417e-02,-0.001412,0.005053,0.000276,0.000708,0.021339,0.002106,0.005152,0.000721
4,0.000534,-0.000307,-4.794044e-06,-0.000049,-0.000040,-0.000532,0.000530,0.000125,-0.004045,0.000528,...,-0.003223,-1.057527e-03,0.000216,0.000674,-0.000147,-0.000276,-0.000144,-0.000144,-0.000144,-0.000144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,-0.186062,-0.311672,0.000000e+00,0.404705,-0.186062,-0.186062,-0.186062,-0.186062,0.004463,-0.186062,...,2.471280,1.896911e-01,0.402004,-1.197746,-0.153630,-0.186062,-0.186062,-0.186062,-0.186062,-0.186062
216,0.000012,-0.000010,-8.112134e-08,0.000008,0.000005,0.000007,0.000012,-0.000008,-0.000018,0.000012,...,0.000013,-3.194137e-07,0.000005,0.000004,0.000010,0.000007,0.000006,0.000007,0.000007,0.000007
217,-0.159441,0.060485,-1.616384e-04,-0.266074,-0.003591,-0.014357,-0.160346,0.430712,-0.087761,-0.003591,...,0.240335,4.736647e-03,0.080431,0.182903,-0.003582,-0.031375,-0.071739,0.290542,-0.003591,-0.003591
218,0.018489,0.068528,-3.170835e-02,-0.032751,0.034595,-0.004508,0.066360,-0.006121,-0.023233,0.067313,...,-0.048174,-1.188590e-01,-0.124145,-0.010733,0.000845,-0.004508,0.001485,-0.004508,-0.004508,0.001485


In [172]:
ridge_feature_rank = abs(ridge_df.mean()).sort_values(ascending=False)
ridge_feature = list(ridge_feature_rank.head(30).index)
ridge_feature

['BTZ',
 'TCA',
 'RMS',
 'BCA',
 'MKP',
 'SO2',
 'OEC',
 'GHN',
 'NOE',
 'BER',
 'BLC',
 'GHP',
 'PSU',
 'NDA',
 'CHA',
 'GHG',
 'RCY',
 'VOE',
 'MHP',
 'TCG',
 'PMD',
 'TCC',
 'FCL',
 'CBP',
 'SOE',
 'NOT',
 'LUF',
 'WWG',
 'TBN',
 'OEB']

#### SHAP Value

In [173]:
from sklearn.linear_model import Lasso
import shap


folder_path_country = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata_new/"
file_names_country = os.listdir(folder_path_country)

# Initialize DataFrame for Shapley results
shap_df = pd.DataFrame(columns=feature)

for file in file_names_country:
    file_path = os.path.join(folder_path_country, file)
    try:
        # Attempt to read the Excel file
        df_country = pd.read_excel(file_path, engine='openpyxl')
    except BadZipFile:
        print(f"Skipping file {file}: Not a valid zip file")
        continue
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    
    country_name = os.path.splitext(file)[0]
    
    for c in methane_df_t.columns:
        if country_name in c:
            y_country = methane_df_t[c].values
            break
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(df_country, y_country, test_size=0.2, random_state=42)
    
    X_train_100 = shap.utils.sample(X_train, 100)  # 100 instances for use as the background distribution

    # A simple linear model
    lasso = Lasso(alpha=0.1,max_iter=10000)
    lasso.fit(X_train, y_train)

    # Explain the model's predictions using SHAP
    explainer = shap.Explainer(lasso, X_train_100)
    shap_values = explainer(X_train) # (number of samples, number of features)
    
    # Compute shap value for each data point and take avg over the whole data set
    # number of avg = number of features

    shap_values_array = shap_values.values 
    column_mean = np.mean(shap_values_array, axis=0)
    sorted_index = np.argsort(column_mean)
    sorted_features = X_train.columns[sorted_index] # feature names
    sorted_column_means = column_mean[sorted_index] # values
    new_row_df = pd.Series(sorted_column_means, index=sorted_features)
    new_row_df = new_row_df.to_frame().T
    shap_df = pd.concat([shap_df, new_row_df], ignore_index=True)

shap_df

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.192e+01, tolerance: 6.187e+00
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.517e+02, tolerance: 7.131e+00
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.480e-01, tolerance: 1.443e-03
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.657e+01, tolerance: 1.868e+01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.995e+01, tolerance: 4.950e+00
Objective did n

,PDN,VOE,CBP,HPE,LED,TC5,LUF,RMS,NXA,FCL,...,CDA_cbrt_diff,GHN_cbrt_diff,SMW_cbrt_diff,WWG_cbrt_diff,WRR_cbrt_diff,NDA_cbrt_diff,BCA_cbrt_diff,GDB_cbrt_diff,SHI_cbrt_diff,CHA_cbrt_diff
0,0.000000e+00,0.0,0.0,NaN,NaN,NaN,0.000000e+00,0.0,0.000000e+00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000e+00,0.0,0.0,0.000000e+00,0.0,5.088522e-17,0.000000e+00,0.0,0.000000e+00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000e+00,0.0,NaN,0.000000e+00,NaN,6.143812e-19,0.000000e+00,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000e+00,0.0,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.000000e+00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0.0,0.000000e+00,NaN,1.445603e-19,0.000000e+00,NaN,0.000000e+00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,NaN,0.0,0.0,4.070818e-16,NaN,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.850372e-17,NaN
216,0.000000e+00,0.0,0.0,0.000000e+00,NaN,NaN,0.000000e+00,0.0,0.000000e+00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,NaN,0.0,0.0,NaN,NaN,NaN,0.000000e+00,0.0,1.850372e-16,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,NaN,0.0,0.0,0.000000e+00,NaN,1.526557e-16,0.000000e+00,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [174]:
shap_feature_rank = abs(shap_df.mean()).sort_values(ascending=False)
shap_feature = list(shap_feature_rank.head(30).index)
shap_feature

['GHG',
 'GPC',
 'PDN',
 'CDO',
 'TBN',
 'POP_cbrt_diff',
 'RMS',
 'TCC',
 'GTI',
 'POP',
 'PSU',
 'PMD',
 'GHN',
 'PFL',
 'BCA',
 'NOX',
 'NOT',
 'OZD',
 'SO2',
 'CDF',
 'SHI',
 'UWD',
 'TCG',
 'USD',
 'SPI',
 'SMW',
 'TKP',
 'WWC',
 'FLI',
 'FSS']

#### Added together

In [177]:
common_elements = set(lasso_feature) & set(ridge_feature) & set(shap_feature)
common_elements

{'BCA', 'GHG', 'GHN', 'PMD', 'SO2', 'TCG'}

Get top 20 features

Get common features
- If > 10 (>7), good
- If not, reduce feature importance methods

- 'BCA', Adjusted emissions growth rate for black carbon
- 'GHG', Greenhouse gas emissions
- 'GHN', GHG Net0 by 2050, Greenhouse Gas Net Emissions
- 'PMD', Ambient particulate matter pollution, Particulate Matter Density
- 'SO2', Sulfur Dioxide
- 'TCG', Net change in tree cover, Total Greenhouse Gases
- 'TCC', Tree cover loss, annual, Total Carbon Content
- 'FOG', F-gases, Fuel Oil Gas

Reasonable story why the common features are important for methane prediction (why relevent)

### Prediction Model

For each country extract the 8 features from the dataset as X to make prediction of methane emission Y.

In [235]:
most_feature = ['BCA', 'GHG', 'GHN', 'PMD', 'SO2', 'TCG', 'TCC', 'FOG']

In [236]:
folder_path_country = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata/"
file_names_country = os.listdir(folder_path_country)

# for each country
for file in file_names_country:
    file_path = os.path.join(folder_path_country, file)
    try:
        # Attempt to read the Excel file
        df_country = pd.read_excel(file_path, engine='openpyxl')
    except BadZipFile:
        print(f"Skipping file {file}: Not a valid zip file")
        continue
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    country_name = os.path.splitext(file)[0]
    
    x_country = pd.DataFrame(columns=most_feature)
    for f in most_feature:
        if f in df_country.columns:
            x = df_country[f].values
            # add x to x_country
            x_country[f] = x
            
    file_path_new = f"/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata_features/{country_name}.xlsx"
    x_country.to_excel(file_path_new, index=False)
                

#### SVM

In [226]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [231]:
methane_df_tt = methane_df.T
methane_df_tt.columns = methane_df_tt.iloc[0]
methane_df_tt = methane_df_tt.drop(methane_df_tt.index[0])
methane_df_tt.head()

country,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
CH4.raw.1992,301.0,119.0,2500.0,3.12,1110.0,0.0935,2.84,4460.0,54.7,0.802,...,767.0,1250.0,12.0,1940.0,2370.0,301.0,301.0,317.0,529.0,436.0
CH4.raw.1993,308.0,124.0,2670.0,3.18,1230.0,0.097,2.89,4450.0,47.4,0.846,...,792.0,1320.0,12.8,1980.0,2450.0,306.0,306.0,348.0,526.0,372.0
CH4.raw.1994,322.0,146.0,2650.0,3.25,1630.0,0.101,2.94,4510.0,47.2,0.893,...,798.0,1440.0,13.0,2060.0,2490.0,301.0,301.0,380.0,522.0,380.0
CH4.raw.1995,341.0,145.0,2920.0,3.33,1820.0,0.104,2.96,4590.0,46.0,0.929,...,793.0,1450.0,13.1,2260.0,2620.0,320.0,320.0,359.0,519.0,387.0
CH4.raw.1996,377.0,138.0,3000.0,3.37,2030.0,0.106,2.95,4470.0,46.2,0.941,...,805.0,1510.0,13.2,2320.0,2700.0,338.0,338.0,377.0,495.0,438.0


In [237]:
folder_path_country = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_countrydata_features/"
file_names_country = os.listdir(folder_path_country)

mse_list = []
r2_list = []

# for each country
for file in file_names_country:
    file_path = os.path.join(folder_path_country, file)
    try:
        # Attempt to read the Excel file
        X = pd.read_excel(file_path, engine='openpyxl')
        X = X.values
    except BadZipFile:
        print(f"Skipping file {file}: Not a valid zip file")
        continue
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue
    country_name = os.path.splitext(file)[0]
    
    for c in methane_df_tt.columns:
        if country_name in c:
            Y = methane_df_tt[c].values
            # print(type(Y))
            break
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler_X = StandardScaler()
    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)

    # Standardize the target variable based on the training data
    scaler_y = StandardScaler()
    y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
    y_test = scaler_y.transform(y_test.reshape(-1, 1)).ravel()  # Standardize y_test for evaluation

    # svm regression
    svm_regressor = SVR()
    
    # Define the parameter grid
    param_grid = {
        'C': [0.1, 1, 10, 100],        # Regularization parameter
        'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
        'gamma': ['scale', 'auto'],    # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
        'degree': [3, 4, 5]            # Degree of the polynomial kernel function (only relevant for 'poly')
    }

    # Use GridSearchCV to search for the best parameters
    grid_search = GridSearchCV(svm_regressor, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)


    # Get the best parameters and the best model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    print("Best Parameters:", best_params)

    # Evaluate the model on the test set
    y_pred = best_model.predict(X_test)
    
    # Train the classifier on the training data
    svm_regressor.fit(X_train, y_train)

    # Predict the labels for the test data
    y_pred = svm_regressor.predict(X_test)

    # Calculate regression metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mse_list.append(mse)
    r2_list.append(r2)
    

    

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 1, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 1, 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf'}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 100, 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf'}
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 100, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
Fitting 5 folds for each of 7

In [238]:
np.mean(mse_list)

0.16466563368657927

In [239]:
np.mean(r2_list)

0.7434691297179317

#### XGB

In [240]:
import optuna
import xgboost as xgb

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <04125532-9495-3051-97BC-F23BE76BA2F9> /Applications/Mu Editor.app/Contents/Resources/Python/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr

In [ ]:
# Define the objective function for Optuna
def xgb_objective(trial):
    # Define hyperparameters to tune
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.7, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.5, 0.7, 0.9, 1.0]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
    }
    
    # Train the model
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)

    # Predict and calculate RMSE
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    return rmse

In [ ]:
# SVM
# XGB
# RF